<a href="https://colab.research.google.com/github/iejjlopez/HolaJJ/blob/master/ProyectoRecomendador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paso 1: Preparación del Entorno
Abra Google Colab.
Cree un nuevo notebook de Python.
Paso 2: Importar Bibliotecas

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162997 sha256=96610d4df4715ada0c7da3135c3328f9d76943e7e027b3f0e5be299df9374dec
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVD, SVDpp, NMF, CoClustering

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el archivo de datos
file_path = '/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv'
data = pd.read_csv(file_path, names=['userId', 'productId', 'rating', 'timestamp'])

# Mostrar el nombre del archivo en la impresión
print(f"Nombres de las columnas en el DataFrame del archivo '{file_path}':")
column_names = data.columns
print(column_names)

# Visualizar los primeros 5 registros del DataFrame
print(f"\nPrimeros 5 registros del DataFrame del archivo '{file_path}':")
print(data.head())

# Estadísticas descriptivas
print(f"\nEstadísticas descriptivas del DataFrame del archivo '{file_path}':")
print(data.describe())

# Sobrescribir el archivo CSV original con los nuevos nombres de columnas
data.to_csv(file_path, index=False)

<ipython-input-4-dae848d45cf3>:8: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, names=['userId', 'productId', 'rating', 'timestamp'])


Nombres de las columnas en el DataFrame del archivo '/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv':
Index(['userId', 'productId', 'rating', 'timestamp'], dtype='object')

Primeros 5 registros del DataFrame del archivo '/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv':
           userId   productId  rating   timestamp
0            user        prod  rating   timestamp
1   AKM1MP6P0OYPR  0132793040     5.0  1365811200
2  A2CX7LUOHB2NDG  0321732944     5.0  1341100800
3  A2NWSAGRHCP8N5  0439886341     1.0  1367193600
4  A2WNBOD3WNDNKT  0439886341     3.0  1374451200

Estadísticas descriptivas del DataFrame del archivo '/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv':
               userId   productId     rating   timestamp
count         7824483     7824483  7824483.0     7824483
unique        4201697      476003       11.0       10013
top     A5JLAU2

In [5]:
import pandas as pd

# Cargar el archivo de datos
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv')

# Verificar los nombres de las columnas
column_names = data.columns
print("Nombres de las columnas en el DataFrame:")
print(column_names)

# Comprobar si 'timestamp' está presente en los nombres de las columnas
if 'timestamp' in column_names:
    print("'timestamp' está presente en el DataFrame.")
else:
    print("La columna 'timestamp' no está presente en el DataFrame. Verifica los nombres de las columnas.")

# Acceder a los primeros registros para visualizar los datos
print("\nPrimeros registros del DataFrame:")
print(data.head())

<ipython-input-5-50b511491a25>:4: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv')


Nombres de las columnas en el DataFrame:
Index(['userId', 'productId', 'rating', 'timestamp'], dtype='object')
'timestamp' está presente en el DataFrame.

Primeros registros del DataFrame:
           userId   productId  rating   timestamp
0            user        prod  rating   timestamp
1   AKM1MP6P0OYPR  0132793040     5.0  1365811200
2  A2CX7LUOHB2NDG  0321732944     5.0  1341100800
3  A2NWSAGRHCP8N5  0439886341     1.0  1367193600
4  A2WNBOD3WNDNKT  0439886341     3.0  1374451200


El código proporcionado implementa un sistema de recomendación de productos utilizando diferentes algoritmos de filtrado colaborativo y análisis de contenido. Los pasos principales y objetivos del código son:

Carga de datos: Se carga el archivo CSV con las calificaciones de los usuarios sobre los productos.

Exploración de datos: Se imprimen los primeros registros del DataFrame, el número de registros y el número de valores faltantes en cada columna.

Preprocesamiento de datos: Se renombran las columnas del DataFrame y se agrupan los datos por usuario y producto.

Filtrado colaborativo: Se implementan diferentes algoritmos de filtrado colaborativo, como KNNBasic, KNNWithMeans, SVD, SVDpp, NMF y CoClustering. Se entrena cada algoritmo con los datos de entrenamiento y se evalúa su rendimiento mediante la puntuación MAP (Mean Average Precision).

Análisis de contenido: Se realiza un análisis de contenido utilizando la biblioteca scikit-learn. Se calcula la matriz de distancias entre los productos utilizando la distancia coseno y se generan recomendaciones de productos para cada usuario. Se evalúa el rendimiento del análisis de contenido mediante la puntuación MAP.

Comparación de resultados: Se comparan los resultados de los diferentes algoritmos de filtrado colaborativo y el análisis de contenido mediante un gráfico de barras que muestra la puntuación MAP para cada algoritmo.

El objetivo principal del código es evaluar y comparar el rendimiento de diferentes algoritmos de recomendación de productos utilizando datos de calificaciones de usuarios y productos.

# Paso 3: Cargar y Preprocesar los Datos

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVD, SVDpp, NMF, CoClustering
from surprise.model_selection import train_test_split
from sklearn.metrics import average_precision_score

file_path = '/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv'
data = pd.read_csv(file_path, names=['userId', 'productId', 'rating', 'timestamp'], dtype=str)

# Seleccionar un subconjunto aleatorio de usuarios
random_users = np.random.choice(data['userId'].unique(), size=100, replace=False)
subset_data = data[data['userId'].isin(random_users)]

# Convertir la columna de timestamp a formato de fecha y hora
subset_data['timestamp'] = pd.to_datetime(subset_data['timestamp'], unit='s')

# Dividir los datos en conjuntos de entrenamiento y prueba
train_start = '2022-01-01'
train_end = '2022-06-30'
test_start = '2022-07-01'
test_end = '2022-12-31'
train_data = subset_data[(subset_data['timestamp'] >= train_start) & (subset_data['timestamp'] <= train_end)]
test_data = subset_data[(subset_data['timestamp'] >= test_start) & (subset_data['timestamp'] <= test_end)]

# Crear un objeto Surprise Dataset a partir de los datos de entrenamiento
reader = Reader(rating_scale=(1, 5))
train_dataset = Dataset.load_from_df(train_data[['userId', 'productId', 'rating']], reader)

# Entrenar y evaluar los algoritmos de filtrado colaborativo
algorithms = [KNNBasic(), KNNWithMeans(), SVD(), SVDpp(), NMF(), CoClustering()]
map_scores = {}

for algorithm in algorithms:
    algorithm.fit(train_dataset.build_full_trainset())
    predictions = algorithm.test(test_data[['userId', 'productId', 'rating']].values)

    if predictions:
        y_true = [pred.r_ui for pred in predictions]
        y_scores = [pred.est for pred in predictions]
        map_score = average_precision_score(y_true, y_scores)
    else:
        map_score = 0.0

    map_scores[algorithm.__class__.__name__] = map_score

# Imprimir los resultados del MAP para cada algoritmo
for algorithm_name, map_score in map_scores.items():
    print(f"{algorithm_name}: {map_score}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNBasic: 0.0
KNNWithMeans: 0.0
SVD: 0.0
SVDpp: 0.0
NMF: 0.0
CoClustering: 0.0


<ipython-input-6-8ec6acfe84f8>:16: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  subset_data['timestamp'] = pd.to_datetime(subset_data['timestamp'], unit='s')
<ipython-input-6-8ec6acfe84f8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['timestamp'] = pd.to_datetime(subset_data['timestamp'], unit='s')
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python

In [ ]:
falta memoria

In [ ]:

# Cargar el archivo CSV
file_path = '/content/drive/MyDrive/Colab Notebooks/ICARO/Proyecto Recomendacion/ratings_Electronics_2.csv'
data = pd.read_csv(file_path, names=['userId', 'productId', 'rating', 'timestamp'], dtype={'userId': str, 'productId': str, 'rating': str, 'timestamp': str})
data = data.dropna(subset=['rating'])  # Eliminar filas con valores nulos en la columna 'rating'
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')  # Convertir la columna 'rating' a numérica
data = data.dropna(subset=['rating'])  # Eliminar filas con valores no numéricos en la columna 'rating'



# Preprocesamiento de datos
data = data.rename(columns={'userId': 'account_id', 'productId': 'content_id', 'rating': 'counts'})
data_grouped = data.groupby(['account_id', 'content_id']).size().reset_index(name='counts')

# Filtrado colaborativo
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(data_grouped[['account_id', 'content_id', 'counts']], reader)
train_data = train_data.build_full_trainset()

# SVDpp
algo_svdpp = SVDpp(n_epochs=5, reg_all=0.4, random_state=42)
algo_svdpp.fit(train_data)
predictions_svdpp = algo_svdpp.test(train_data.build_testset())
map_svdpp = average_precision_score(list(zip(*predictions_svdpp))[0], list(zip(*predictions_svdpp))[3])

# SVD
algo_svd = SVD()
algo_svd.fit(train_data)
predictions_svd = algo_svd.test(train_data.build_testset())
map_svd = average_precision_score(list(zip(*predictions_svd))[0], list(zip(*predictions_svd))[3])

# NMF
algo_nmf = NMF()
algo_nmf.fit(train_data)
predictions_nmf = algo_nmf.test(train_data.build_testset())
map_nmf = average_precision_score(list(zip(*predictions_nmf))[0], list(zip(*predictions_nmf))[3])

# CoClustering
algo_coclustering = CoClustering()
algo_coclustering.fit(train_data)
predictions_coclustering = algo_coclustering.test(train_data.build_testset())
map_coclustering = average_precision_score(list(zip(*predictions_coclustering))[0], list(zip(*predictions_coclustering))[3])

# Análisis de contenido
tfidf = TfidfVectorizer(stop_words='english')
content_tfidf = tfidf.fit_transform(data_grouped['content_id'])
distances = pairwise_distances(content_tfidf, metric='cosine')
n_neighbors = 10
nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(content_tfidf)
distances, indices = nbrs.kneighbors(content_tfidf)
content_recommendations = []
for i in range(len(indices)):
    content_recommendations.append(data_grouped['content_id'][indices[i][1:]])
map_contenido = average_precision_score([data_grouped['content_id'][i] for i in data_grouped['account_id']], content_recommendations)

# Comparación de resultados
map_scores = [map_svdpp, map_svd, map_nmf, map_coclustering, map_contenido]
algorithms = ['SVDpp', 'SVD', 'NMF', 'CoClustering', 'Análisis de Contenido']
plt.figure(figsize=(10, 6))
plt.bar(algorithms, map_scores, color='skyblue')
plt.xlabel('Algoritmo')
plt.ylabel('MAP')
plt.title('Comparación de MAP entre algoritmos')
plt.show()